<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração
Aqui nós extraimos os dados da internet dos carros

In [ ]:
import requests
import re
import pandas as pd

## Extrair todos os cógidos FIPE

Feito manualmente, todos os códigos FIPE estão presentes na pagina:
https://www.tabelafipebrasil.com/fipe/carros
bastou fazer um regex no formato [\d]+-[\d]


arquivo gerado ficou diisponível em "https://github.com/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/FIPE-CODES.txt"

In [ ]:
def get_fipe_codes():
  codes_string = requests.get("https://raw.githubusercontent.com/JoaoVitorSantiagoNogueira/case-brick-seguros/main/FIPE-CODES.txt").text
  codes_list = re.split("\r\n",codes_string)
  return codes_list

In [ ]:
len(get_fipe_codes())

6893

## Pegar Informações de Veiculo

In [ ]:
# https://brasilapi.com.br/api/fipe/preco/v1/038003-2  Últimos números são o código fipe

def get_json_from_fipe(Codigo_FIPE, query = ""):

  if query != "":
    query = "?tabela_referencia="+query

  response = requests.get("https://brasilapi.com.br/api/fipe/preco/v1/" + Codigo_FIPE + query)
  return response.json()


In [ ]:
#sample
get_json_from_fipe("038003-2", "5")

{'message': 'Tabela de referência inválida',
 'type': 'bad_request',
 'name': 'BadRequestError'}

#Cria a tabela inteira de um veiculo
Navegando preenchendo mês-a-mês, no formato mm-aa

## Cria o data frame base

In [ ]:
df = pd.json_normalize(get_json_from_fipe("038003-2", "10"))
df

,valor,marca,modelo,anoModelo,combustivel,codigoFipe,mesReferencia,tipoVeiculo,siglaCombustivel,dataConsulta
0,"R$ 18.742,00",Acura,Integra GS 1.8,1992,Gasolina,038003-2,março de 2006,1,G,"sexta-feira, 10 de maio de 2024 21:18"
1,"R$ 17.197,00",Acura,Integra GS 1.8,1991,Gasolina,038003-2,março de 2006,1,G,"sexta-feira, 10 de maio de 2024 21:18"


In [ ]:
#While mes referencia != ultimo, faz a busca.
def complete_dataset_prices(df):
  for index, row in df.iterrows():

